### Bienvenido a la Semana 5 Día 1

¡AutoGen AgentChat!

Esto debería parecer simple y familiar, porque tiene mucho en común con Crew y el SDK de OpenAI Agents.

In [16]:
from dotenv import load_dotenv
load_dotenv(override=True)

ModuleNotFoundError: No module named 'dotenv'

### Primer concepto: el Modelo

In [2]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

In [4]:
from autogen_ext.models.ollama import OllamaChatCompletionClient
ollamamodel_client = OllamaChatCompletionClient(model="llama3.2")

### Segundo concepto: el Mensaje

In [5]:
from autogen_agentchat.messages import TextMessage
message = TextMessage(content="Me gustaría ir a Londres", source="user")
message

TextMessage(id='bbbccf5d-bc40-46c3-8cfa-532132c30f08', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2026, 1, 8, 14, 9, 42, 861158, tzinfo=datetime.timezone.utc), content='Me gustaría ir a Londres', type='TextMessage')

### Tercer concepto: el Agente

In [8]:
from autogen_agentchat.agents import AssistantAgent

agent = AssistantAgent(
    name="airline_agent",
    model_client=model_client,
    system_message="Eres un asistente útil para una aerolínea. Das respuestas breves y con humor.",
    model_client_stream=True
)

### Ponemos todo junto con on_messages

In [9]:
from autogen_core import CancellationToken

response = await agent.on_messages([message], cancellation_token=CancellationToken())
response.chat_message.content

'¡Genial! Londres tiene un clima único... Eso que llamamos "lluvia mágica". ¿Listo para hacer tu maleta y disfrutar de una taza de té mientras esperas al sol? ☔️✈️'

### Hagamos una base de datos local de precios de entradas

In [10]:
import os
import sqlite3

# Eliminar el archivo de base de datos existente si existe
if os.path.exists("tickets.db"):
    os.remove("tickets.db")

# Creamos la base de datos y la tabla
conn = sqlite3.connect("tickets.db")
c = conn.cursor()
c.execute("CREATE TABLE cities (city_name TEXT PRIMARY KEY, round_trip_price REAL)")
conn.commit()
conn.close()

In [11]:
# Rellenamos la base de datos
def save_city_price(city_name, round_trip_price):
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("REPLACE INTO cities (city_name, round_trip_price) VALUES (?, ?)", (city_name.lower(), round_trip_price))
    conn.commit()
    conn.close()

# Some cities!
save_city_price("Londres", 299)
save_city_price("París", 399)
save_city_price("Roma", 499)
save_city_price("Madrid", 550)
save_city_price("Barcelona", 580)
save_city_price("Berlín", 525)

In [12]:
import sqlite3

# Método para obtener el precio de un vuelo a una ciudad
def get_city_price(city_name: str) -> float | None:
    """Obtener el precio de un vuelo a una ciudad"""
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("SELECT round_trip_price FROM cities WHERE city_name = ?", (city_name.lower(),))
    result = c.fetchone()
    conn.close()
    return result[0] if result else None

In [13]:
get_city_price("Londres")

299.0

In [14]:
from autogen_agentchat.agents import AssistantAgent

smart_agent = AssistantAgent(
    name="smart_airline_agent",
    model_client=model_client,
    system_message="Eres un asistente útil para una aerolínea. Das respuestas breves y con humor. Puedes usar la herramienta get_city_price para obtener el precio de un vuelo a una ciudad.",
    model_client_stream=True,
    tools=[get_city_price],
    reflect_on_tool_use=True
)

In [15]:
response = await smart_agent.on_messages([message], cancellation_token=CancellationToken())
for inner_message in response.inner_messages:
    print(inner_message.content)
response.chat_message.content

'¡Fantástico! Londres tiene mucho que ofrecer: desde los majestuosos palacios hasta el mejor té. ¿Te gustaría saber el precio de un vuelo a Londres?'